# Notes

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from imblearn.pipeline import Pipeline 
from sklearn.metrics import mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/JOB-HUNTING/CV_NOW/Accenture_REPL_DS/exercise

/content/drive/MyDrive/JOB-HUNTING/CV_NOW/Accenture_REPL_DS/exercise


In [5]:
hourly_sales = pd.read_csv('data/hourly_sales_features.csv')
orig_columns = hourly_sales.columns

type conversion: convert string to datetime

In [6]:
hourly_sales.hour = pd.to_datetime(hourly_sales.hour)


In [7]:
print(min(hourly_sales.hour))
print(max(hourly_sales.hour))

2013-11-07 13:00:00
2018-05-06 19:00:00


In [8]:
hourly_sales.head()

,hour,num_of_hours,num_of_weeks,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,AR__sales_7_weeks_ago_p_0h,seasonal__hod,seasonal__dow,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas
0,2013-11-07 13:00:00,0,0,3457.0,NaN,NaN,NaN,NaN,NaN,NaN,13,3,False,False,False,False,False,False,False,False,48.0
1,2013-11-07 14:00:00,1,0,2250.0,NaN,NaN,NaN,NaN,NaN,NaN,14,3,False,False,False,False,False,False,False,False,48.0
2,2013-11-07 15:00:00,2,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,15,3,False,False,False,False,False,False,False,False,48.0
3,2013-11-07 16:00:00,3,0,729.0,NaN,NaN,NaN,NaN,NaN,NaN,16,3,False,False,False,False,False,False,False,False,48.0
4,2013-11-07 17:00:00,4,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,17,3,False,False,False,False,False,False,False,False,48.0


In [9]:
hourly_sales.shape

(39391, 21)

In [10]:
hourly_sales.columns

Index(['hour', 'num_of_hours', 'num_of_weeks', 'sales',
       'AR__sales_weekly_total_52weeks_ago',
       'AR__sales_weekly_total_51weeks_ago', 'AR__sales_364_DAYS_ago_same_hod',
       'AR__sales_5_weeks_ago_p_0h', 'AR__sales_6_weeks_ago_p_0h',
       'AR__sales_7_weeks_ago_p_0h', 'seasonal__hod', 'seasonal__dow',
       'holiday__is_easter', 'holiday__is_thanksgiving',
       'holiday__is_thanksgiving_post_1d', 'holiday__is_thanksgiving_post_2d',
       'holiday__is_thanksgiving_post_3d', 'holiday__is_christmas',
       'holiday__is_christmas_post_1d',
       'holiday__between_thanksgiving_n_christmas',
       'holiday__days_to_next_christmas'],
      dtype='object')

# Feature data transformation

remove data without full features because we use AR features 1 year ago.

In [11]:
print(hourly_sales.shape)
begin_time = hourly_sales.hour.iloc[0]
hourly_sales = hourly_sales[hourly_sales.hour > begin_time+pd.Timedelta(370,'d')]
print(hourly_sales.shape)

(39391, 21)
(30510, 21)


## keep features & labels only

In [12]:
hourly_sales.set_index('num_of_hours',inplace = True) # useful for validation and testing
hourly_sales.drop(columns = ['num_of_weeks'], axis = 1, inplace = True) # keep column 'hour' for train test split

In [13]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,AR__sales_7_weeks_ago_p_0h,seasonal__hod,seasonal__dow,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas
num_of_hours,,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,3086.0,14,2,False,False,False,False,False,False,False,False,43.0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,2435.0,15,2,False,False,False,False,False,False,False,False,43.0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,3083.0,16,2,False,False,False,False,False,False,False,False,43.0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,3146.0,17,2,False,False,False,False,False,False,False,False,43.0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,2304.0,18,2,False,False,False,False,False,False,False,False,43.0


## Null Value check

In [14]:
pd.set_option('display.max_rows', None)
hourly_sales.isna().sum()

hour                                         0
sales                                        0
AR__sales_weekly_total_52weeks_ago           0
AR__sales_weekly_total_51weeks_ago           0
AR__sales_364_DAYS_ago_same_hod              0
AR__sales_5_weeks_ago_p_0h                   0
AR__sales_6_weeks_ago_p_0h                   0
AR__sales_7_weeks_ago_p_0h                   0
seasonal__hod                                0
seasonal__dow                                0
holiday__is_easter                           0
holiday__is_thanksgiving                     0
holiday__is_thanksgiving_post_1d             0
holiday__is_thanksgiving_post_2d             0
holiday__is_thanksgiving_post_3d             0
holiday__is_christmas                        0
holiday__is_christmas_post_1d                0
holiday__between_thanksgiving_n_christmas    0
holiday__days_to_next_christmas              0
dtype: int64

## Categorical & boolean Feature Transformation

In [15]:
hourly_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30510 entries, 8881 to 39390
Data columns (total 19 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   hour                                       30510 non-null  datetime64[ns]
 1   sales                                      30510 non-null  float64       
 2   AR__sales_weekly_total_52weeks_ago         30510 non-null  float64       
 3   AR__sales_weekly_total_51weeks_ago         30510 non-null  float64       
 4   AR__sales_364_DAYS_ago_same_hod            30510 non-null  float64       
 5   AR__sales_5_weeks_ago_p_0h                 30510 non-null  float64       
 6   AR__sales_6_weeks_ago_p_0h                 30510 non-null  float64       
 7   AR__sales_7_weeks_ago_p_0h                 30510 non-null  float64       
 8   seasonal__hod                              30510 non-null  int64         
 9   seasonal__dow 

In [16]:
# one-hot encoding for categorical features
categorical_featuers = [feature for feature in list(hourly_sales.columns) if 'seasonal' in feature]
print(categorical_featuers)
print(hourly_sales.shape)
for feature in categorical_featuers:
  hourly_sales = pd.concat(
      [hourly_sales.drop(feature,axis=1),
      pd.get_dummies(hourly_sales[feature],
                      prefix=feature+'_')],
                      axis=1)
print(hourly_sales.shape)

['seasonal__hod', 'seasonal__dow']
(30510, 19)
(30510, 48)


In [17]:
# convert booelan to int
boolean_features = [feature for feature in list(hourly_sales.columns) if 'holiday' in feature]
if 'holiday__days_to_next_christmas' in boolean_features:
  boolean_features.remove('holiday__days_to_next_christmas')
for feature in boolean_features:
  print(feature)
print(hourly_sales.shape)
for feature in boolean_features:
  hourly_sales[feature] = hourly_sales[feature].astype(int)
print(hourly_sales.shape)

holiday__is_easter
holiday__is_thanksgiving
holiday__is_thanksgiving_post_1d
holiday__is_thanksgiving_post_2d
holiday__is_thanksgiving_post_3d
holiday__is_christmas
holiday__is_christmas_post_1d
holiday__between_thanksgiving_n_christmas
(30510, 48)
(30510, 48)


# Split dataset to train, validation and test

Data is very valuable in this dataset, I split the dataset into only train, test sets, rather than train, dev, test sets.

In [18]:
hourly_sales.shape

(30510, 48)

In [19]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,AR__sales_7_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__1,seasonal__hod__2,seasonal__hod__3,seasonal__hod__4,seasonal__hod__5,seasonal__hod__6,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,3086.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,2435.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,3083.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,3146.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,2304.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [20]:
# remove hourly data not with full day from the last day
print(hourly_sales.hour.iloc[-1])
hourly_sales = hourly_sales[hourly_sales.hour <  pd.to_datetime(hourly_sales.hour.iloc[-1].date()) ]
print(hourly_sales.hour.iloc[-1])

2018-05-06 19:00:00
2018-05-05 23:00:00


In [21]:
total_days = (hourly_sales.hour.iloc[-1] - hourly_sales.hour.iloc[0]).days
print('total_days: ', total_days)
print('15% days: ', int(total_days * 0.15))
test_days = 180
train_days = total_days - test_days
print('used train_days: ', train_days)
# print('used dev_days: ', dev_days)
print('used test_days: ', test_days)

total_days:  1270
15% days:  190
used train_days:  1090
used test_days:  180


In [22]:
train_data = hourly_sales[hourly_sales.hour <= hourly_sales.hour.iloc[-1]-pd.Timedelta(test_days,'d') ]
test_data = hourly_sales[hourly_sales.hour > hourly_sales.hour.iloc[-1]-pd.Timedelta(test_days,'d' )]
print('train data range')
print(train_data.hour.iloc[0])
print(train_data.hour.iloc[-1])
print()
print('test data range')
print(test_data.hour.iloc[0])
print(test_data.hour.iloc[-1])

train data range
2014-11-12 14:00:00
2017-11-06 23:00:00

test data range
2017-11-07 00:00:00
2018-05-05 23:00:00


In [23]:
train_data.head()


,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,AR__sales_7_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__1,seasonal__hod__2,seasonal__hod__3,seasonal__hod__4,seasonal__hod__5,seasonal__hod__6,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,3086.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,2435.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,3083.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,3146.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,2304.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [24]:
# replicate holidays  (same effect of oversampling) and test the performance
# christmas
# christmas_rows = train_data[(train_data.hour.dt.month==12) & (train_data.hour.dt.day==25)]
# train_data = train_data.append([christmas_rows]*1000, ignore_index=False)

In [25]:
print('test_date has {} days'.format(test_data.shape[0]/24))

test_date has 180.0 days


In [26]:
train_data_x = train_data.drop(columns = ['hour','sales'], axis = 1)
train_data_y = train_data['sales']

test_data_x = test_data.drop(columns = ['hour','sales'], axis = 1)
test_data_y = test_data['sales']

In [27]:
print(train_data_x.shape)
print(train_data_y.shape)
print(test_data_x.shape)
print(test_data_y.shape)

(26170, 46)
(26170,)
(4320, 46)
(4320,)


# Model Selection

The following models are trained: logistic regression, random froest. If we want to model trend with non-linear models, probably neural network is better.

In [28]:
performance = pd.DataFrame({'train':[None]*3,'test': [None]*3, 'test_real_time': [None]*3})
performance.index = ['LR', 'RF','BT']
performance

,train,test,test_real_time
LR,None,None,None
RF,None,None,None
BT,None,None,None


In [29]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,AR__sales_7_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__1,seasonal__hod__2,seasonal__hod__3,seasonal__hod__4,seasonal__hod__5,seasonal__hod__6,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,3086.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,2435.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,3083.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,3146.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,2304.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


## Training set performance

In [30]:
regressors = [
    LinearRegression(),
    # DecisionTreeClassifier(),
    RandomForestRegressor()
    # GradientBoostingRegressor(),
    ]

Train model using training data

In [31]:
scaler = StandardScaler()
scaler.fit(train_data_x)
train_data_x_scaled = scaler.transform(train_data_x)

In [32]:
# def build_ml_pipeline(regressor):
#   steps = []
#   steps.append(('fillnan', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.0)))
#   steps.append(('scaler',StandardScaler())) #scaler applies only to training data, not validation data to avoid data leakage
#   steps.append(('model', regressor))
#   pipeline = Pipeline(steps = steps)
#   return pipeline

### linear regression

In [33]:
#-----linear regression
regressor = Ridge() #alpha=1.0  Ridge
# regressor = LinearRegression()#alpha=1.0  Ridge
# training model using train_set
regressor.fit(train_data_x_scaled,train_data_y)
# performance on training set
y_predicted = regressor.predict(train_data_x_scaled)
rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
print(regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
linear_regressor = regressor
performance.loc['LR','train'] = rmse

Ridge :RMSE is (419.976)


In [34]:
# fig, ax = plt.subplots()
# markerline1, stemlines1, baseline = plt.stem(range(len(train_data_y)),train_data_y, 'b',markerfmt = 'bo', label = 'truth')
# plt.setp(stemlines1, 'linewidth', 0.5,'markerfacecolor', 'b')

# markerline2, stemlines2, baseline = plt.stem(range(len(y_predicted)), y_predicted, 'r-.',markerfmt = 'rx', label = 'predicted')
# plt.setp(stemlines2, 'linewidth', 0.5,'markerfacecolor', 'r')
# fig = plt.gcf()
# fig.set_size_inches(46, 9)
# plt.xlim([-1,100])
# plt.show()

### Random forest

In [35]:
#-----RF regression
regressor = RandomForestRegressor() # GradientBoostingRegressor RandomForestRegressor
# training model using train_set
regressor.fit(train_data_x_scaled,train_data_y)
# performance on training set
y_predicted = regressor.predict(train_data_x_scaled)
rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
print(regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
RF_regressor = regressor
performance.loc['RF','train'] = rmse

RandomForestRegressor :RMSE is (134.001)


In [36]:
# fig, ax = plt.subplots()
# markerline1, stemlines1, baseline = plt.stem(range(len(train_data_y)),train_data_y, 'b',markerfmt = 'bo', label = 'truth')
# plt.setp(stemlines1, 'linewidth', 0.5,'markerfacecolor', 'b')

# markerline2, stemlines2, baseline = plt.stem(range(len(y_predicted)), y_predicted, 'r-.',markerfmt = 'rx', label = 'predicted')
# plt.setp(stemlines2, 'linewidth', 0.5,'markerfacecolor', 'r')
# fig = plt.gcf()
# fig.set_size_inches(46, 9)
# plt.xlim([-1,100])
# plt.show()

### Bossted trees

In [37]:
#-----RF regression
regressor = GradientBoostingRegressor() # GradientBoostingRegressor RandomForestRegressor
# training model using train_set
regressor.fit(train_data_x_scaled,train_data_y)
# performance on training set
y_predicted = regressor.predict(train_data_x_scaled)
rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
print(regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
boosted_regressor = regressor
performance.loc['BT','train'] = rmse

GradientBoostingRegressor :RMSE is (360.598)


## Feature imoprtance

### Importance LR

In [38]:
import math
coeff = np.squeeze(linear_regressor.coef_)
importance = abs(np.array(coeff))/np.sum(abs(np.array(coeff)))*100
sort_index = np.argsort(importance)
data = {'features':np.array(train_data_x.columns)[sort_index[::-1]], 'importance':importance[sort_index[::-1]] }
pd.DataFrame(data)

,features,importance
0,AR__sales_364_DAYS_ago_same_hod,13.940810
1,AR__sales_5_weeks_ago_p_0h,4.201214
2,AR__sales_7_weeks_ago_p_0h,3.895072
3,AR__sales_6_weeks_ago_p_0h,3.791217
4,seasonal__hod__13,3.343492
5,seasonal__hod__14,3.321864
6,seasonal__hod__12,3.265424
7,seasonal__hod__15,3.178545
8,seasonal__hod__16,3.104606
9,seasonal__hod__11,3.009924


### Importance RF

In [39]:
importances = RF_regressor.feature_importances_
feature_names = train_data_x.columns.tolist()

In [40]:
df_importances = pd.DataFrame(
    {"feature":feature_names, "importance":importances}).sort_values("importance", ascending=False)
# top15_features = df_importances   
# top15_features
df_importances.importance = df_importances.importance *100
df_importances.reset_index(drop=True)

,feature,importance
0,AR__sales_364_DAYS_ago_same_hod,88.610189
1,AR__sales_7_weeks_ago_p_0h,2.286748
2,AR__sales_5_weeks_ago_p_0h,1.766441
3,holiday__days_to_next_christmas,1.344659
4,AR__sales_6_weeks_ago_p_0h,1.218469
5,AR__sales_weekly_total_51weeks_ago,0.584563
6,AR__sales_weekly_total_52weeks_ago,0.581989
7,holiday__is_christmas,0.528010
8,seasonal__hod__21,0.523423
9,holiday__is_easter,0.350314


## Test set performance

### linear regression

In [41]:
test_data_x_scaled = scaler.transform(test_data_x)

In [42]:
#-----linear regression
# performance on training set
y_predicted = linear_regressor.predict(test_data_x_scaled)
rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(linear_regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
performance.loc['LR','test'] = rmse

Ridge :RMSE is (481.459)


### Random Froest

In [43]:
#-----RF
y_predicted = RF_regressor.predict(test_data_x_scaled)
rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(RF_regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
performance.loc['RF','test'] = rmse

RandomForestRegressor :RMSE is (403.513)


### Boosted Trees

In [44]:

y_predicted = boosted_regressor.predict(test_data_x_scaled)
rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(RF_regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
performance.loc['BT','test'] = rmse

RandomForestRegressor :RMSE is (413.787)


# Hyper parameter tuning

## Random Froest

In [45]:
n_estimators_vector = [50,100,150]# [50,100,150]
min_samples_leaf_vector = [1,2,3] #[1,2,3]
min_samples_split_vector = [2,3,4] #[2,3,4]
n1,n2,n3 = len(n_estimators_vector), len(min_samples_leaf_vector), len(min_samples_split_vector)
RF_performance = pd.DataFrame(index = range(n1*n2*n3), columns = ['n_estimators','min_samples_leaf','min_samples_split','train_rmse','test_rmse'])

i  = 0
for n_estimators in n_estimators_vector:
  for min_samples_leaf in min_samples_leaf_vector:
    for min_samples_split in min_samples_split_vector:
      print(i)
      #-----training
      regressor = RandomForestRegressor(n_estimators = n_estimators, min_samples_leaf = min_samples_leaf,min_samples_split = min_samples_split)
      regressor.fit(train_data_x_scaled,train_data_y)
      y_predicted = regressor.predict(train_data_x_scaled)
      train_rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
      #-----testing
      y_predicted = regressor.predict(test_data_x_scaled)
      test_rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS

      RF_performance.iloc[i,:] = [n_estimators,min_samples_leaf,min_samples_split,train_rmse,test_rmse]
      i += 1
RF_performance.sort_values('test_rmse')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


,n_estimators,min_samples_leaf,min_samples_split,train_rmse,test_rmse
18,150,1,2,132.9164,400.193007
19,150,1,3,141.32628,400.787308
20,150,1,4,153.448398,400.95774
11,100,1,4,154.072647,401.166918
21,150,2,2,182.077672,401.873038
9,100,1,2,133.796179,402.48031
10,100,1,3,142.881119,402.902137
13,100,2,3,183.551898,402.933594
1,50,1,3,145.779572,403.013591
12,100,2,2,182.442588,403.205188


# Real-time test set performance

In this section, the performance is evaluated on the data chunk of every 30 days. This is to evaluate the actual performance when predicting hourly sales/traffic for the next month. 

 The following auto regressive features' values needs to be updated in real-time manner, meaning that when a sale value for certain time(hour) is predicted. The newly predicted sale value should be used as features rather than using the truth data. This is related to the fact that we need to predict sales 30 days ahead. 

In [46]:
update_features = [ 'AR__sales_1_weeks_ago_m_1h',
                    'AR__sales_1_weeks_ago_p_0h',
                    'AR__sales_1_weeks_ago_p_1h',
                    'AR__sales_2_weeks_ago_m_1h',
                    'AR__sales_2_weeks_ago_p_0h',
                    'AR__sales_2_weeks_ago_p_1h',
                    'AR__sales_3_weeks_ago_m_1h',
                    'AR__sales_3_weeks_ago_p_0h',
                    'AR__sales_3_weeks_ago_p_1h',
                    'AR__sales_4_weeks_ago_m_1h',
                    'AR__sales_4_weeks_ago_p_0h',
                    'AR__sales_4_weeks_ago_p_1h',
                    'AR__sales_1_days_ago_m_1h',
                    'AR__sales_1_days_ago_p_0h',
                    'AR__sales_1_days_ago_p_1h',
                    'AR__sales_2_days_ago_m_1h',
                    'AR__sales_2_days_ago_p_0h',
                    'AR__sales_2_days_ago_p_1h',
                    'AR__sales_3_days_ago_m_1h',
                    'AR__sales_3_days_ago_p_0h',
                    'AR__sales_3_days_ago_p_1h',
                    'AR__sales_4_days_ago_m_1h',
                    'AR__sales_4_days_ago_p_0h',
                    'AR__sales_4_days_ago_p_1h',
                    'AR__sales_pre_1_h',
                    'AR__sales_pre_2_h',
                    'AR__sales_pre_3_h',
                    'AR__sales_pre_4_h',
                    'AR__sales_pre_5_h',
                    'AR__sales_pre_6_h',
                    'AR__sales_pre_7_h',
                    'AR__sales_pre_8_h',
                    'AR__sales_pre_9_h',
                    'AR__sales_pre_10_h',
                    'AR__sales_pre_11_h',
                    'AR__sales_pre_12_h',
                    'AR__sales_pre_13_h',
                    'AR__sales_pre_14_h',
                    'AR__sales_pre_15_h',
                    'AR__sales_pre_16_h',
                    'AR__sales_pre_17_h',
                    'AR__sales_pre_18_h',
                    'AR__sales_pre_19_h',
                    'AR__sales_pre_20_h',
                    'AR__sales_pre_21_h',
                    'AR__sales_pre_22_h']
update_features_copy = update_features.copy()
update_features = []
for feature in update_features_copy:
  if feature in test_data_x.columns:
    update_features.append(feature)
update_features

[]

## Random forest

In [47]:
# number of months in test data
num_months_test = int(test_data_y.shape[0]/24/30)

we will udpate the table 'aux_hourly_sales' in real time when evaluating the model performance, note we need to copy the dataframe again for every 30 days to mimic the real practice

In [48]:
#-----RF regression
regressor = RandomForestRegressor(n_estimators = 150, min_samples_leaf = 1, min_samples_split = 3) 
# training model using train_set
regressor.fit(train_data_x_scaled,train_data_y)
# performance on training set
y_predicted = regressor.predict(train_data_x_scaled)
rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
print(regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
RF_regressor = regressor
performance.loc['RF','train'] = rmse

RandomForestRegressor :RMSE is (141.275)


In [49]:
from utility import calc_row_index_hourly
y_predicted = np.empty((test_data_y.shape[0],),dtype = float)

for j in range(num_months_test): # iterate over each 30 days
  aux_hourly_sales = hourly_sales.copy(deep = True)
  # for i in range(test_data_x.shape[0]):
  for i in range(j*30*24, (j+1)*30*24): # iterate over jth month(30 days)
    # update AR features subject to change from possibly newly predicted data, because this problem is for predicting 1 month ahead
    hour_index = test_data_x.index[i] # hour index for current sales to be predicted
    features_cur_row = test_data_x.iloc[i,:].copy(deep = True)
    for feature in update_features:
      features_cur_row[feature] = aux_hourly_sales.loc[hour_index - calc_row_index_hourly(feature), 'sales']
    # scale data 
    scaled_features_cur_row = scaler.transform(features_cur_row.values.reshape(1,-1))
    # inference 
    y_predicted[i] = RF_regressor.predict(scaled_features_cur_row)
    # update sale value in real time
    aux_hourly_sales.loc[hour_index,'sales'] = y_predicted[i]

rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(RF_regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
performance.loc['RF','test_real_time'] = rmse

RandomForestRegressor :RMSE is (400.557)


In [50]:
# fig, ax = plt.subplots()
# markerline1, stemlines1, baseline = plt.stem(range(len(test_data_y)),test_data_y, 'b',markerfmt = 'bo', label = 'truth')
# plt.setp(stemlines1, 'linewidth', 0.5,'markerfacecolor', 'b')

# markerline2, stemlines2, baseline = plt.stem(range(len(y_predicted)), y_predicted, 'r-.',markerfmt = 'rx', label = 'predicted')
# plt.setp(stemlines2, 'linewidth', 0.5,'markerfacecolor', 'r')
# fig = plt.gcf()
# fig.set_size_inches(46, 9)
# plt.xlim([0,400])
# plt.show()

# Save data for traffic data prediction features

In [51]:
print(min(hourly_sales.hour))
print(max(hourly_sales.hour))

2014-11-12 14:00:00
2018-05-05 23:00:00


In [52]:
train_data.to_csv('data/train_data_sales.csv')

test_data_for_predicting_traffic = test_data[['hour']]
test_data_for_predicting_traffic['predicted_sales'] = y_predicted
test_data_for_predicting_traffic.to_csv('data/test_data_sales.csv')

In [53]:
test_data_for_predicting_traffic.head()

,hour,predicted_sales
num_of_hours,,
35051,2017-11-07 00:00:00,0.0
35052,2017-11-07 01:00:00,0.0
35053,2017-11-07 02:00:00,0.0
35054,2017-11-07 03:00:00,0.0
35055,2017-11-07 04:00:00,0.0


# Performance summary table

In [54]:
print('Features used:')
orig_columns

Features used:


Index(['hour', 'num_of_hours', 'num_of_weeks', 'sales',
       'AR__sales_weekly_total_52weeks_ago',
       'AR__sales_weekly_total_51weeks_ago', 'AR__sales_364_DAYS_ago_same_hod',
       'AR__sales_5_weeks_ago_p_0h', 'AR__sales_6_weeks_ago_p_0h',
       'AR__sales_7_weeks_ago_p_0h', 'seasonal__hod', 'seasonal__dow',
       'holiday__is_easter', 'holiday__is_thanksgiving',
       'holiday__is_thanksgiving_post_1d', 'holiday__is_thanksgiving_post_2d',
       'holiday__is_thanksgiving_post_3d', 'holiday__is_christmas',
       'holiday__is_christmas_post_1d',
       'holiday__between_thanksgiving_n_christmas',
       'holiday__days_to_next_christmas'],
      dtype='object')

In [55]:
performance

,train,test,test_real_time
LR,419.97572,481.458507,None
RF,141.274899,403.513296,400.55696
BT,360.597624,413.786638,None


In [56]:
RF_performance.sort_values('test_rmse').iloc[0:1,:]

,n_estimators,min_samples_leaf,min_samples_split,train_rmse,test_rmse
18,150,1,2,132.9164,400.193007


Not making too much difference after adding 'AR__sales_1_years_ago_same_hod'

Compared with 'model_AR_longRangeOnly', although this notebook achievs better test performance because of more mid/short term AR features were used, the generlizd real-time performance is the comparable with  'model_AR_longRangeOnly'. Because mid/short term AR features degrades the performance in real-time. **So short/mid term AR feastures is not really very useful for real-time data.**

In [57]:
result = pd.DataFrame({'predicted':y_predicted,'truth': test_data_y})
result['hour'] = test_data.hour
result.to_csv('data/result.csv')

In [58]:
y_predicted_corrected = y_predicted
y_predicted_corrected[y_predicted<0] = 0

rmse = mean_squared_error(test_data_y, y_predicted_corrected, squared=False) # RMS
print(regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)

RandomForestRegressor :RMSE is (400.557)
